## Trabajo Práctico NLP: Detección de Tópicos y Clasificación

**Mariela Iaccarino**

Certificación Experta en NLP - ITBA

#### Este código está diseñado para procesar tópicos diariamente, almacenarlos en una base de datos, comparar tópicos entre días y decidir si un tópico es nuevo o si debe mergearse con uno existente.

El flujo será el siguiente:

1) Entrenar el modelo usando solo los datos del primer día.
2) Guardar los tópicos generados en la base de datos.
3) Procesar los datos de los días siguientes, comparando los nuevos tópicos con los existentes en la base de datos.
4) Decidir si un tópico es nuevo o si debe mergearse con uno existente.
5) Generar nuevos tópicos si no hay coincidencias relevantes.
6) Generar Inferencia y Clasificación de Nuevos Documentos

## 1. Configuración y Carga de Datos


1.1 Importación de librerías necesarias:

Primero, se instalan e importan las librerías necesarias para el análisis:

In [1]:
!pip install datasets umap-learn chromadb hdbscan sentence_transformers BERTopic opensearch-py matplotlib

In [2]:
!pip install opensearch-py==2.3.0

In [3]:
from datasets import load_dataset
import pandas as pd
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from datetime import datetime
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
from opensearch_data_model import Topic, TopicKeyword, os_client
from dateutil.parser import parse
from utils import SPANISH_STOPWORDS
import numpy as np
from datetime import timedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)


/home/RECONQUISTA/a118672/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
Topic.init()

2. Configuración del Modelo de Embeddings y Otros Modelos:

Utilizaremos BERTopic para la detección de tópicos. Aplicaremos UMAP para la reducción de dimensionalidad y HDBSCAN para el clustering de los embeddings.

In [5]:
# Configuración del modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Configuración de UMAP para reducción de dimensionalidad
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Configuración de HDBSCAN para clustering
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)




In [6]:
# Configuración de pipelines para NER y análisis de sentimiento
ner_pipeline = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


3. Funciones de Utilidad


In [7]:


# Función para extracción de entidades
def extract_entities(text):
    entities = ner_pipeline(text)
    return [entity['word'] for entity in entities]

# Función para análisis de sentimiento
def analyze_sentiment(text):
    sentiment = sentiment_pipeline(text)
    return sentiment[0]


In [8]:
def calculate_similarity_threshold(topic_id, topic_model, probs):
    topic_probs = [probs[i] for i, topic in enumerate(topic_model.topics_) if topic == topic_id]
    mean_prob = np.mean(topic_probs)
    std_dev_prob = np.std(topic_probs)
    threshold = mean_prob + 1.5 * std_dev_prob
    return threshold


In [9]:
def extract_entities(text):
    entities = ner_pipeline(text)
    return entities

def analyze_sentiment(text):
    sentiment = sentiment_pipeline(text)
    return sentiment


In [10]:
# Función para generar el nombre del tópico
def get_topic_name(keywords):
    # Verificar si 'keywords' es una lista
    if isinstance(keywords, list) and len(keywords) > 0:
        # Verificar la estructura del primer elemento de 'keywords'
        first_element = keywords[0]

        # Si el primer elemento es una tupla o lista, intentamos extraer el primer elemento de cada tupla/lista
        if isinstance(first_element, (tuple, list)):
            try:
                # Intentamos descomponer asumiendo que hay al menos dos elementos en la tupla/lista
                return ', '.join([k[0] for k in keywords[:4]])
            except (IndexError, ValueError):
                # Si hay un problema al descomponer, devolvemos las keywords como están
                return ', '.join([str(k) for k in keywords[:4]])
        else:
            # Si el primer elemento no es una tupla/lista, asumimos que es una lista de strings
            return ', '.join([str(k) for k in keywords[:4]])
    else:
        return "Tópico Desconocido"




In [11]:
def topic_threshold(topic_id, topic_model, probs):
    try:
        docs_per_topics =  [i for i, x in enumerate(topic_model.topics_) if x== topic_id    ]
        return np.array(  [probs[doc_idx] for doc_idx in docs_per_topics  ]).mean()
    except:
        return 0

In [12]:
def delete_index_opensearch(index_name: str) -> bool:

    try:
        # Consulta para eliminar todos los documentos
        delete_query = {
                        "query": {
                        "match_all": {}
                        }
        }

        # Ejecutar la operación de borrado por consulta
        response = os_client.delete_by_query(index=index_name, body=delete_query)

        return True

    except Exception as e:
        print(f"Ha ocurrido un error: {e}")
        return

3. Generación del Datase

In [13]:
"""
# Cargar datasets y agregar el campo de fecha
def add_fecha_field(dataset, fecha):
    df = pd.DataFrame(dataset['train'])
    df['date'] = fecha
    return df

datasets = [
    ("jganzabalseenka/news_2024-07-01_24hs", '2024-07-01'),
    ("jganzabalseenka/news_2024-07-12_24hs", '2024-07-12'),
    ("jganzabalseenka/news_2024-07-14_24hs", '2024-07-14'),
    ("jganzabalseenka/news_2024-07-16_24hs", '2024-07-16'),
    ("jganzabalseenka/news_2024-07-19_24hs", '2024-07-19')
]

df_list = [add_fecha_field(load_dataset(ds[0]), ds[1]) for ds in datasets]
df = pd.concat(df_list, ignore_index=True)


"""

'\n# Cargar datasets y agregar el campo de fecha\ndef add_fecha_field(dataset, fecha):\n    df = pd.DataFrame(dataset[\'train\'])\n    df[\'date\'] = fecha\n    return df\n\ndatasets = [\n    ("jganzabalseenka/news_2024-07-01_24hs", \'2024-07-01\'),\n    ("jganzabalseenka/news_2024-07-12_24hs", \'2024-07-12\'),\n    ("jganzabalseenka/news_2024-07-14_24hs", \'2024-07-14\'),\n    ("jganzabalseenka/news_2024-07-16_24hs", \'2024-07-16\'),\n    ("jganzabalseenka/news_2024-07-19_24hs", \'2024-07-19\')\n]\n\ndf_list = [add_fecha_field(load_dataset(ds[0]), ds[1]) for ds in datasets]\ndf = pd.concat(df_list, ignore_index=True)\n\n\n'

In [14]:

# Cargar datasets y agregar el campo de fecha
def add_fecha_field(dataset, fecha, sample_size=500):
    df = pd.DataFrame(dataset['train'])
    df['date'] = fecha
    # Tomar una muestra aleatoria de 1000 registros si el dataset tiene más de 1000 registros
    if len(df) > sample_size:
        df = df.sample(n=sample_size, random_state=42)
    return df

datasets = [
    ("jganzabalseenka/news_2024-07-01_24hs", '2024-07-01'),
    ("jganzabalseenka/news_2024-07-12_24hs", '2024-07-12'),
    ("jganzabalseenka/news_2024-07-14_24hs", '2024-07-14'),
    ("jganzabalseenka/news_2024-07-16_24hs", '2024-07-16'),
    ("jganzabalseenka/news_2024-07-19_24hs", '2024-07-19')
]

df_list = [add_fecha_field(load_dataset(ds[0]), ds[1], sample_size=500) for ds in datasets]
df = pd.concat(df_list, ignore_index=True)

# Verificar la cantidad de registros cargados
print(f"Total de registros cargados: {len(df)}")


Total de registros cargados: 2500


In [15]:
print(f"Total de registros cargados: {len(df)}")

Total de registros cargados: 2500


## 2. Procesamiento y Detección de Tópicos Diarios

Cada día, procesaremos los textos y determinaremos los tópicos. Luego, compararemos estos tópicos con los días anteriores para decidir si se deben mergear con tópicos existentes o si deben crearse como nuevos.

### Paso 1: Procesamiento de Datos del Primer Día

In [16]:
delete_index_opensearch("topic")

True

In [17]:
def process_first_day(df):
    # Definir el vectorizador usando todas las entidades y keywords únicas del dataset
    entities = set(sum([list(e) for e in df['entities_transformers'].values], []))
    keywords = set(sum([list(e) for e in df['keywords'].values], []))
    all_tokens = list(entities.union(keywords))

    # Configurar el vectorizador
    tf_vectorizer = CountVectorizer(
        ngram_range=(1, 3),
        stop_words=SPANISH_STOPWORDS,
        lowercase=False,
        vocabulary=all_tokens,
    )

    # Configurar UMAP y HDBSCAN para el modelo BERTopic
    umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
    hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

    # Configurar el modelo BERTopic
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=tf_vectorizer,
        ctfidf_model=ClassTfidfTransformer(),
        representation_model=KeyBERTInspired(),
        language='spanish'
    )

    # Entrenar el modelo con los textos del primer día
    topics, probs = topic_model.fit_transform(df['text'].tolist())

    # Generar embeddings de los textos usando el modelo de embeddings
    texts = df['text'].tolist()
    embeddings = topic_model.embedding_model.embed(texts)
    sim_matrix = cosine_similarity(topic_model.topic_embeddings_, embeddings)

    for topic in topic_model.get_topics().keys():
        if topic > -1:
            print(f"Procesando tópico {topic}...")

            keywords = topic_model.get_topic(topic)
            topic_keywords = [TopicKeyword(name=k, score=s) for k, s in keywords]
            threshold = topic_threshold(topic, topic_model, probs)
            from_date = parse(df['date'].min())
            to_date = from_date + timedelta(days=1)

            # Verificar que topic + 1 esté dentro del rango válido
            if topic + 1 < sim_matrix.shape[0]:
                best_doc_index = sim_matrix[topic + 1].argmax()
            else:
                best_doc_index = sim_matrix[topic].argmax()

            best_doc = df.iloc[best_doc_index].text

            topic_doc = Topic(
                vector=list(topic_model.topic_embeddings_[topic]),
                similarity_threshold=threshold,
                created_at=datetime.now(),
                to_date=to_date,
                from_date=from_date,
                index=topic,
                keywords=topic_keywords,
                name=get_topic_name(keywords),
                best_doc=best_doc
            )

            # Intentar guardar el tópico y manejar errores si ocurren
            try:
                result = topic_doc.save()
                print(f"Tópico guardado con éxito: {result}")
            except Exception as e:
                print(f"Error al guardar el tópico: {str(e)}")


In [18]:
# Procesar primer día
first_day_df = df[df['date'] == df['date'].min()]
process_first_day(first_day_df)


/home/RECONQUISTA/a118672/.local/lib/python3.8/site-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df) + 1)


Procesando tópico 0...
Tópico guardado con éxito: created
Procesando tópico 1...
Tópico guardado con éxito: created


In [19]:
Topic.search().count()

0

### Paso 2: Merge de Tópicos entre Días

Para evitar la proliferación de tópicos redundantes, implementaremos un criterio de agrupación de tópicos que se aplica dentro de un mismo día y entre días diferentes. Usaremos cosine similarity para realizar la comparación:

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datetime import datetime
from dateutil.parser import parse

def merge_topics(new_topic_embedding, existing_topics, new_topic_threshold=0.75):
    for existing_topic in existing_topics:
        similarity = cosine_similarity([new_topic_embedding], [existing_topic.vector])[0][0]
        if similarity > new_topic_threshold:
            return existing_topic
    return None

def adjust_similarity_threshold(initial_threshold, current_topic_count, desired_topic_count):
    if current_topic_count > desired_topic_count:
        return min(initial_threshold + 0.01, 0.95)
    elif current_topic_count < desired_topic_count:
        return max(initial_threshold - 0.01, 0.65)
    else:
        return initial_threshold


### Paso 3: Procesamiento de Días Siguientes

Ahora procesamos los días siguientes, comparando los tópicos con los ya existentes y decidiendo si mergear o crear un nuevo tópico.

In [21]:
from joblib import Parallel, delayed
import numpy as np

def process_subsequent_days(df, date, initial_threshold=0.75, adjust_threshold=False):
    # Generar embeddings de todos los textos a la vez para optimizar el proceso
    embeddings = embedding_model.encode(df['text'].tolist(), show_progress_bar=True)

    # Convertir los embeddings a una lista de arrays para que pandas los acepte
    df['embeddings'] = [embedding for embedding in embeddings]

    # Cargar todos los tópicos existentes desde OpenSearch solo una vez
    existing_topics = [hit for hit in Topic.search().source(["vector", "keywords"]).execute()]

    current_threshold = initial_threshold
    new_topics_created = False

    for i, row in df.iterrows():
        print(f"Procesando documento {i + 1}/{len(df)}...")

        # Usar la lista de tópicos existentes
        existing_topic = merge_topics(row['embeddings'], existing_topics, current_threshold)

        if existing_topic:
            # Fusión de tópicos
            existing_topic.vector = np.mean([existing_topic.vector, row['embeddings']], axis=0).tolist()
            existing_keywords = set([kw.name for kw in existing_topic.keywords])
            new_keywords = set(row['keywords'])
            combined_keywords = list(existing_keywords.union(new_keywords))
            existing_topic.keywords = [TopicKeyword(name=k, score=1.0) for k in combined_keywords]

            try:
                existing_topic.save()
                print(f"Tópico existente actualizado.")
            except Exception as e:
                print(f"Error al guardar el tópico existente: {str(e)}")
        else:
            # Creación de un nuevo tópico
            topic_name = get_topic_name(row['keywords'])
            new_topic_doc = Topic(
                vector=row['embeddings'].tolist(),
                similarity_threshold=current_threshold,
                created_at=datetime.now(),
                to_date=datetime.now(),
                from_date=datetime.strptime(date, '%Y-%m-%d'),
                index=i,
                name=topic_name,
                best_doc=row['text'],
                keywords=[TopicKeyword(name=k, score=1.0) for k in row['keywords']],
            )

            try:
                new_topic_doc.save()
                new_topics_created = True
                existing_topics.append(new_topic_doc)  # Añadir el nuevo tópico a la lista existente
                print(f"Nuevo tópico guardado con éxito.")
            except Exception as e:
                print(f"Error al guardar el nuevo tópico: {str(e)}")

        print(f"Número de tópicos hasta ahora: {len(existing_topics)}")

    # Ajustar el threshold si no se han creado nuevos tópicos
    if not new_topics_created and adjust_threshold:
        current_threshold *= 0.95  # Reducir el threshold para permitir la creación de nuevos tópicos
        print(f"Ajustando el threshold de similitud a: {current_threshold}")
        process_subsequent_days(df, date, current_threshold)

# Simular procesamiento de días siguientes
for date in df['date'].unique()[1:]:
    print(f"\nProcesando tópicos para la fecha: {date}")
    daily_df = df[df['date'] == date]
    process_subsequent_days(daily_df, date, initial_threshold=0.85, adjust_threshold=True)

# Mostrar el número total de tópicos al final
final_topic_count = Topic.search().count()
print(f"Número total final de tópicos en OpenSearch: {final_topic_count}")



Procesando tópicos para la fecha: 2024-07-12


Batches: 100%|██████████| 16/16 [00:17<00:00,  1.09s/it]


Procesando documento 501/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 3
Procesando documento 502/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 4
Procesando documento 503/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 5
Procesando documento 504/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 6
Procesando documento 505/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 7
Procesando documento 506/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 8
Procesando documento 507/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 9
Procesando documento 508/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 10
Procesando documento 509/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 11
Procesando documento 510/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 12
Procesando docume

Batches: 100%|██████████| 16/16 [00:17<00:00,  1.07s/it]


Procesando documento 1001/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 11
Procesando documento 1002/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 12
Procesando documento 1003/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 13
Procesando documento 1004/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 14
Procesando documento 1005/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 15
Procesando documento 1006/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 16
Procesando documento 1007/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 17
Procesando documento 1008/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 18
Procesando documento 1009/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 19
Procesando documento 1010/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 20


Batches: 100%|██████████| 16/16 [00:17<00:00,  1.10s/it]


Procesando documento 1501/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 11
Procesando documento 1502/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 12
Procesando documento 1503/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 13
Procesando documento 1504/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 14
Procesando documento 1505/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 15
Procesando documento 1506/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 16
Procesando documento 1507/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 17
Procesando documento 1508/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 18
Procesando documento 1509/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 19
Procesando documento 1510/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 20


Batches: 100%|██████████| 16/16 [00:18<00:00,  1.13s/it]


Procesando documento 2001/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 11
Procesando documento 2002/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 12
Procesando documento 2003/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 13
Procesando documento 2004/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 14
Procesando documento 2005/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 15
Procesando documento 2006/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 16
Procesando documento 2007/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 17
Procesando documento 2008/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 18
Procesando documento 2009/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 19
Procesando documento 2010/500...
Nuevo tópico guardado con éxito.
Número de tópicos hasta ahora: 20


In [22]:
Topic.search().count()

1895

## 3. Clasificación de Nuevos Documentos

In [23]:
def classify_text(title, text):
    # Combinar título y texto
    combined_text = title + " " + text

    # Generar embeddings del texto combinado
    new_embed = embedding_model.encode([combined_text])  # Asegúrate de usar encode() si es SentenceTransformer

    # Construir la consulta KNN para OpenSearch
    query = {
        "size": 5,
        "query": {
            "knn": {
                "vector": {
                    "vector": new_embed[0].tolist(),
                    "k": 1000
                }
            }
        }
    }

    # Ejecutar la búsqueda en OpenSearch
    response = os_client.search(index='topic', body=query)

    # Verificar si se encontraron resultados
    if response['hits']['hits']:
        # Obtener la información del tópico más cercano
        topic_id = response['hits']['hits'][0]['_id']
        keywords = response['hits']['hits'][0]['_source']['keywords']
        name = response['hits']['hits'][0]['_source']['name']  # Obtener el nombre del tópico
        best_doc = response['hits']['hits'][0]['_source']['best_doc']  # Obtener el mejor documento

        # Extraer entidades y análisis de sentimiento
        entities = extract_entities(combined_text)
        sentiment = analyze_sentiment(combined_text)

        # Devolver los resultados
        return topic_id, name, keywords, best_doc, entities, sentiment

    # Si no se encuentra ningún resultado, devolver None
    return None, None, None, None, None, None


# Ejemplo de clasificación
new_title = "Argentina Campeon"
new_text = "Termino la copa America y Argentina salio campeon"
topic_id, name, keywords, best_doc, entities, sentiment = classify_text(new_title, new_text)
print(f"Tópico: {topic_id}, Nombre: {name}, Keywords: {keywords}, Best Doc: {best_doc}, Entidades: {entities}, Sentimiento: {sentiment}")


Tópico: 684_eliminatorias,_selección_argentina,_periodista,_partidos, Nombre: eliminatorias, selección argentina, periodista, partidos, Keywords: [{'name': 'eliminatorias', 'score': 1.0}, {'name': 'selección argentina', 'score': 1.0}, {'name': 'periodista', 'score': 1.0}, {'name': 'partidos', 'score': 1.0}, {'name': 'finalistas', 'score': 1.0}, {'name': 'el equipo', 'score': 1.0}, {'name': 'expectativas', 'score': 1.0}, {'name': 'ventaja', 'score': 1.0}, {'name': 'aire', 'score': 1.0}, {'name': 'altura', 'score': 1.0}, {'name': 'campeón', 'score': 1.0}, {'name': 'conductor', 'score': 1.0}, {'name': 'diálogo', 'score': 1.0}, {'name': 'decisión', 'score': 1.0}], Best Doc: En diálogo con el periodista Luis Mino, Gonzalo Bonadeo contó "que el equipo que más estuvo a la altura de sus propias expectativas fue Colombia y aun por arriba de la Selección Argentina".

Embed

Gonzalo Bonadeo y la Copa América: "Hay una ventaja para Argentina"

En otra parte de la entrevista entre Gonzalo Bonadeo, 

In [24]:

# Ejemplo de clasificación
new_title = "Frio"
new_text = "Bajas temperatura en la Patagonia para la próxima semana"
topic_id, name, keywords, best_doc, entities, sentiment = classify_text(new_title, new_text)
print(f"Tópico: {topic_id}, Nombre: {name}, Keywords: {keywords}, Best Doc: {best_doc}, Entidades: {entities}, Sentimiento: {sentiment}")

Tópico: 947_temperaturas_mínimas,_invierno,_estación,_décadas, Nombre: temperaturas mínimas, invierno, estación, décadas, Keywords: [{'name': 'temperaturas mínimas', 'score': 1.0}, {'name': 'invierno', 'score': 1.0}, {'name': 'estación', 'score': 1.0}, {'name': 'décadas', 'score': 1.0}, {'name': 'alto valle', 'score': 1.0}, {'name': 'heladas', 'score': 1.0}, {'name': 'trigo', 'score': 1.0}, {'name': 'máximas', 'score': 1.0}, {'name': 'frío', 'score': 1.0}, {'name': 'bolsa', 'score': 1.0}, {'name': 'registros', 'score': 1.0}, {'name': 'alertas', 'score': 1.0}, {'name': 'comercio', 'score': 1.0}, {'name': 'daños', 'score': 1.0}, {'name': 'diversas zonas', 'score': 1.0}, {'name': 'servicio meteorológico', 'score': 1.0}, {'name': 'historia reciente', 'score': 1.0}], Best Doc: El invierno de 2024 en Argentina se perfila como uno de los más fríos de los últimos 60 años, según la Bolsa de Comercio de Rosario. Las bajas temperaturas, que ya se están haciendo sentir en gran parte del país, podr

In [25]:
# Ejemplo de clasificación
new_title = "Caso Loan"
new_text = "Nilo residente en nueve de julio Corrientes desaparecido"
topic_id, name, keywords, best_doc, entities, sentiment = classify_text(new_title, new_text)
print(f"Tópico: {topic_id}, Nombre: {name}, Keywords: {keywords}, Best Doc: {best_doc}, Entidades: {entities}, Sentimiento: {sentiment}")

Tópico: 1937_la_desaparición,_corrientes,_el_ministro,_niño, Nombre: la desaparición, corrientes, el ministro, niño, Keywords: [{'name': 'la desaparición', 'score': 1.0}, {'name': 'corrientes', 'score': 1.0}, {'name': 'el ministro', 'score': 1.0}, {'name': 'niño', 'score': 1.0}, {'name': 'reglamentaciones', 'score': 1.0}, {'name': 'suposiciones', 'score': 1.0}, {'name': 'peña', 'score': 1.0}, {'name': 'certezas', 'score': 1.0}, {'name': 'paraje rural', 'score': 1.0}, {'name': 'indagatoria', 'score': 1.0}, {'name': 'comisario detenido', 'score': 1.0}, {'name': 'protocolos', 'score': 1.0}, {'name': 'paradero', 'score': 1.0}, {'name': 'jueza federal', 'score': 1.0}, {'name': 'hipótesis', 'score': 1.0}, {'name': 'el caso', 'score': 1.0}, {'name': 'siniestro vial', 'score': 1.0}, {'name': 'testimonio', 'score': 1.0}, {'name': 'matrimonio', 'score': 1.0}, {'name': 'cobertura', 'score': 1.0}, {'name': 'chico', 'score': 1.0}, {'name': 'novedades', 'score': 1.0}, {'name': 'leyes', 'score': 1.0}

## Resumen

Este código integra todas las funcionalidades:

* **Procesamiento diario de tópicos:** Los tópicos se detectan y procesan diariamente, almacenándolos en la base de datos.

* **Comparación de tópicos entre días:** Los tópicos se comparan con los existentes para determinar si deben mergearse o si deben crearse como nuevos.

* **Clasificación de nuevos documentos:** Cuando se recibe un nuevo documento, se compara con los tópicos existentes y se clasifica se extraen entidades y keywords, y se realiza un análisis de sentimiento.

* **Almacenamiento en OpenSearch:** Los tópicos y sus embeddings se almacenan en OpenSearch, permitiendo búsquedas eficientes y comparaciones entre días.